# Récupération des matchs manquants

Suite au merge des datasets nous avons remarqué qu'il manquait un grand nombre de d'observations. 

Certain matchs n'étaient pas dans les datasets play by play. Il est possible de les récupérer via l'API nba_api afin de les réinjecter dans notre pre processing

In [ ]:
# Matchs qui n'apparaissent que dans Shot locations
game_ids = df_merged[df_merged['GAME_ID'].isna()]['Game ID'].unique()
len(game_ids)

1714

In [ ]:
from nba_api.stats.endpoints import playbyplayv2
from tqdm import tqdm

# pour chaque game id de Shot locations non présent dans les play by play
# on récupère les play by play sur le site de la NBA
liste_df = []
for id in tqdm(game_ids):
    string_id = "00" + str(int(id))
    liste_df.append(playbyplayv2.PlayByPlayV2(string_id).get_data_frames()[0])


100%|██████████| 1714/1714 [31:03<00:00,  1.09s/it]


In [ ]:
# concaténation
missing_play_by_play = pd.concat(liste_df)

In [ ]:
# enregistrement du dataset
missing_play_by_play.to_csv("../data/raw/missing_pbp.csv")

# Récupération des stats manquants

In [5]:
import pandas as pd
BEST_PLAYERS =['Kobe Bryant', 'LeBron James', 'Stephen Curry', 'Kevin Durant', 'Dwyane Wade', 
               'Dirk Nowitzki', 'Tim Duncan', "Shaquille O'Neal", "Steve Nash", "Kawhi Leonard", 
               "James Harden", "Jason Kidd", "Allen Iverson", "Chris Webber", "Kevin Garnett", 
               "Paul Pierce", "Giannis Antetokounmpo", "Jimmy Butler", "Russell Westbrook", "Dwight Howard"]


In [56]:
dfs = []
for i in range(2018,2021):
    df = pd.read_csv('../data/raw/season_stats_'+str(i)+'.csv')
    df_advanced = pd.read_csv('../data/raw/season_stats_advanced_'+str(i)+'.csv')
    df['Year']=i
    df = df.merge(df_advanced, on=['Player', 'Tm','Pos', 'Age', 'G', 'MP'])
    dfs.append(df)

stats = pd.concat(dfs)

In [57]:
stats = stats[stats.Player.isin(BEST_PLAYERS)]

In [58]:
stats.columns

Index(['Rk_x', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Player-additional_x', 'Year', 'Rk_y', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Player-additional_y'],
      dtype='object')

In [60]:
# Change column order to fit existing data
stats = stats.reindex(['Year','Player','Pos','Age','Tm','G','GS','MP',
                       'PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%',
                       'BLK%','TOV%','USG%', 'Unnamed: 19','OWS','DWS','WS','WS/48','Unnamed: 24',
                       'OBPM','DBPM','BPM','VORP','FG','FGA','FG%','3P','3PA','3P%',
                       '2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB',
                       'AST','STL','BLK','TOV','PF','PTS'], axis=1)
    

In [61]:
stats.to_csv('../data/raw/seasons_stats_2018-2020.csv')